In [11]:
#Imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from io import StringIO
from typing import List, Set, IO

In [12]:
# Set up Selenium
service: Service = Service()
options: webdriver.ChromeOptions = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no GUI)
options.add_argument('--no-sandbox')  # Bypass OS security model, needed for running in Docker
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
driver: webdriver.Chrome = webdriver.Chrome(service=service, options=options)

In [13]:
# URL to scrape
url: str = "https://www.fangraphs.com/standings/playoff-odds"

In [14]:
# Navigate to the page
driver.get(url)

In [15]:
# Wait for the tables to load
wait: WebDriverWait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'playoff-odds-table')))

[<selenium.webdriver.remote.webelement.WebElement (session="09513687e24ee9b72dd071f81bd7663f", element="f.A75D8D6A98B977C78848A521E02B8553.d.6CFD85B813059E8DE2122AE05B32C8AC.e.79")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09513687e24ee9b72dd071f81bd7663f", element="f.A75D8D6A98B977C78848A521E02B8553.d.6CFD85B813059E8DE2122AE05B32C8AC.e.80")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09513687e24ee9b72dd071f81bd7663f", element="f.A75D8D6A98B977C78848A521E02B8553.d.6CFD85B813059E8DE2122AE05B32C8AC.e.81")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09513687e24ee9b72dd071f81bd7663f", element="f.A75D8D6A98B977C78848A521E02B8553.d.6CFD85B813059E8DE2122AE05B32C8AC.e.82")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09513687e24ee9b72dd071f81bd7663f", element="f.A75D8D6A98B977C78848A521E02B8553.d.6CFD85B813059E8DE2122AE05B32C8AC.e.83")>,
 <selenium.webdriver.remote.webelement.WebElement (session="09513687e24ee9b72dd071f81

In [16]:
# Get the page source and parse with BeautifulSoup
soup: BeautifulSoup = BeautifulSoup(driver.page_source, 'html.parser')

In [17]:
# Find all tables with class "playoff-odds-table"
tables: List[BeautifulSoup] = soup.find_all('table', class_='playoff-odds-table')

In [18]:
# Close the Selenium driver
driver.quit()

In [39]:
for table in unique_tables:
    html_str: str = str(table)
    html_io: IO[str] = StringIO(html_str)
    dfs = pd.read_html(html_io)
    df_list.extend(dfs)